# Algorithm for detect wave

1. We need MACD, open, close
2. Looking where MACD change sign (two points begin and end of wave). Backward in time
3. Looking only for negative waves
4. Stop searching after n_waves (for example 3)
5. Inside wave we know the value of the lowes price and the lowest MACD
6. Check if:
6.1 For new wave price has to be lower. Previous value multiplied by coefficient
6.2 For new wave MACD value hat to be bigger (less in absolute). Previous value multiplied by coefficient
7.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import talib.abstract as ta
import numba

In [2]:
columns=['date', 'open', 'high', 'low', 'close','volume']
df_json = pd.read_json("../data/binance/ALGO_USDT-1m.json")

# time open high low close volume
df_json.columns = columns

In [3]:
# macd = ta.MACD(df_json, fastperiod=7, slowperiod=15 , signalperiod=7)
macd = ta.MACD(df_json)
df_json['macdhist'] = macd['macdhist']
df_json['low_value'] = df_json[['open', 'close']].min(axis = 1)

In [6]:
print(len(df_json), len(df_json) / 60 / 24 / 365)
df_json


918006 21.25013888888889


,date,open,high,low,close,volume,macdhist,low_value
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,NaN,0.4465
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,NaN,0.4467
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,NaN,0.4464
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,NaN,0.4460
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,NaN,0.4466
...,...,...,...,...,...,...,...,...
918001,1664944440000,0.3525,0.3525,0.3524,0.3525,5655.00,-0.000060,0.3525
918002,1664944500000,0.3524,0.3525,0.3524,0.3525,3878.00,-0.000047,0.3524
918003,1664944560000,0.3525,0.3526,0.3525,0.3526,18512.00,-0.000027,0.3525
918004,1664944620000,0.3527,0.3528,0.3527,0.3527,3693.00,-0.000005,0.3527


## Predict new bollinger

In [4]:
# example how to use delta_abs
# df['delta_abs'] = df['close'].rolling(window=timewindow).apply(delta_abs, raw=True)

@numba.jit(nopython=True)
def delta_abs(values):
    return (values.max() - values.min()) * 100 / values.min()
    # return dataframe['close'].rolling(n).apply(lambda s: (s.max() - s.min()) * 100 / s.min())

start_time = datetime.now()
df_json['delta_abs'] = df_json['close'].rolling(window=200).apply(delta_abs, raw=True)
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 1048.397 ms


In [5]:
start_time = datetime.now()
df_json['delta_abs11'] = df_json['close'].rolling(200).apply(lambda s : (s.max() - s.min()) * 100 / s.min())
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")


Function took 49152.856 ms


In [6]:
df_json

,date,open,high,low,close,volume,macdhist,low_value,delta_abs,delta_abs11
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,NaN,0.4465,NaN,NaN
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,NaN,0.4467,NaN,NaN
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,NaN,0.4464,NaN,NaN
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,NaN,0.4460,NaN,NaN
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,NaN,0.4466,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
918001,1664944440000,0.3525,0.3525,0.3524,0.3525,5655.00,-0.000060,0.3525,0.908059,0.908059
918002,1664944500000,0.3524,0.3525,0.3524,0.3525,3878.00,-0.000047,0.3524,0.908059,0.908059
918003,1664944560000,0.3525,0.3526,0.3525,0.3526,18512.00,-0.000027,0.3525,0.908059,0.908059
918004,1664944620000,0.3527,0.3528,0.3527,0.3527,3693.00,-0.000005,0.3527,0.908059,0.908059


In [ ]:
# write rolling for custom function


In [23]:
sigma_1m = 3
timewindow_for_predict = 11

In [4]:

start_time = datetime.now()
df_json['average_1m'] = df_json['close'].rolling(timewindow_for_predict).apply(lambda s :
                        s.mean() - s.std(ddof=1) * (timewindow_for_predict+1) * (timewindow_for_predict+1 - sigma_1m**2 - 1)**(-0.5))
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 47389.403 ms


In [28]:
@numba.jit(nopython=True)
def average(values, timewindow, sigma):
    n = len(values)
    if n == 1:
        return 0.
    else:
        # return np.mean(values) - np.std(values) * np.sqrt(n / (n - 1)) * (timewindow + 1) * (timewindow + 1 - sigma**2 - 1) ** (-0.5)
        return np.mean(values) - np.std(values) * np.sqrt(n / (n - 1)) * ((timewindow + 1) * sigma ** 2 * (timewindow - sigma ** 2)) ** 0.5

        # return np.mean(values) - np.std(values) * (timewindow + 1) * (timewindow + 1 - sigma**2 - 1)**(-0.5)

In [8]:
start_time = datetime.now()
df_json['average_1m_other'] = [average(df_sub.values, timewindow=timewindow_for_predict, sigma=sigma_1m) for df_sub in df_json['close'].rolling(timewindow_for_predict)]
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 11367.589 ms


In [29]:
start_time = datetime.now()
df_json['average'] = df_json['close'].rolling(window=timewindow_for_predict).apply(average, args=(timewindow_for_predict, sigma_1m), raw=True)
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 504.95599999999996 ms


In [17]:
df_json.size

8262054

In [10]:
def predict_price_boundary_old(df_average, df_close, start=1, stop=11):

    print('sizes', df_average.size, df_close.size)

    if df_average.size < stop:
        return 0
    # start = 1
    # stop = 12

    # k_5 = 2.8
    # value_5 = 0
    # average_5 = 0
    # sigma = 0
    sum_of_close_values = 0

    for i in range(start, stop + 1):
        sum_of_close_values += df_close.iat[-i]

    average = (sum_of_close_values + df_average) / (stop + 1 - start)

    local_sum = 0
    for i in range(start, stop + 1):
        local_sum += (df_close.iat[-i] - average) ** 2
        local_sum += (df_average - average) ** 2

    sigma = (local_sum / (stop + 1 - start)) ** 0.5

    # value_5 = average_5 - sigma*((k_5**2)*stop/(stop-k_5**2-1))**(1/2)

    prediction = ((stop + 1) *
                  ((average - df_average) / sigma) ** 2 /
                  (stop + ((average - df_average) / sigma) ** 2)) ** 0.5

    # prediction = 0

    return prediction.iat[-1]

@numba.jit(nopython=True)
def predict_boundary(data, start=1, stop=11):

    arr_average = data[:, 0]
    arr_close = data[:, 1]

    # arr_average = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] -> [11]
    # arr_close = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] -> [11]

    if arr_average.size < stop:
        return [0, arr_close[-1]]


    x_0 = arr_average[-1]  # average value
    s = stop + 1  # number of points WITH prediction

    close_mean = np.mean(arr_close)
    close_sq_mean = np.mean(np.power(arr_close, 2))
    sigma = np.power((close_sq_mean - np.power(close_mean, 2)) / (s - 1), 0.5)
    deviation = np.power((close_mean - x_0) / sigma, 2)

    prediction = np.power(((s - 1) * deviation) / (s + deviation), 0.5)
    # print(prediction)

    # # k_5 = 2.8
    # # value_5 = 0
    # # average_5 = 0
    # # sigma = 0
    # sum_of_close_values = 0
    #
    # for i in range(start, stop + 1):
    #     sum_of_close_values += arr_close[-i]
    #
    # average = (sum_of_close_values + arr_average[-1]) / (stop + 1 - start)
    #
    # local_sum = 0
    # for i in range(start, stop + 1):
    #     local_sum += (arr_close[-i] - average) ** 2
    #     local_sum += (arr_average[-1] - average) ** 2
    #
    # sigma = (local_sum / (stop + 1 - start)) ** 0.5
    #
    # # value_5 = average_5 - sigma*((k_5**2)*stop/(stop-k_5**2-1))**(1/2)
    #
    # prediction = ((stop + 1) *
    #               ((average - arr_average[-1]) / sigma) ** 2 /
    #               (stop + ((average - arr_average[-1]) / sigma) ** 2)) ** 0.5

    return [prediction, arr_close[-1]]


@numba.jit(nopython=True)
def predict_boundary_old(arr_average, arr_close, start=1, stop=11):

    # arr_average = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] -> [11]
    # arr_close = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] -> [11]

    if arr_average.size < stop:
        return 0

    # start = 1
    # stop = 12

    # k_5 = 2.8
    # value_5 = 0
    # average_5 = 0
    # sigma = 0
    sum_of_close_values = 0

    for i in range(start, stop + 1):
        sum_of_close_values += arr_close[-i]

    average = (sum_of_close_values + arr_average[-1]) / (stop + 1 - start)

    local_sum = 0
    for i in range(start, stop + 1):
        local_sum += (arr_close[-i] - average) ** 2
        local_sum += (arr_average[-1] - average) ** 2

    sigma = (local_sum / (stop + 1 - start)) ** 0.5

    # value_5 = average_5 - sigma*((k_5**2)*stop/(stop-k_5**2-1))**(1/2)

    prediction = ((stop + 1) *
                  ((average - arr_average[-1]) / sigma) ** 2 /
                  (stop + ((average - arr_average[-1]) / sigma) ** 2)) ** 0.5

    return prediction

In [186]:
# @numba.jit(nopython=True)
def test(df):
    # print('iteration')
    print(df, df[-1][0], df[-1][1], df[:, 0])
    # print(df[-1][1])
    return df[-1][0] * df[-1][1], df[-1][1]

def test2(value1):
    # print('iteration')
    print(value1)
    # a = df['close']
    return 0

In [79]:
df_json_small = df_json[:100000]
df_json_small

,date,open,high,low,close,volume,macdhist,low_value,test_dw_1,test_dw_2,average,predict
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,NaN,0.4465,0,NaN,NaN,NaN
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,NaN,0.4467,0,NaN,NaN,NaN
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,NaN,0.4464,0,NaN,NaN,NaN
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,NaN,0.4460,0,NaN,NaN,NaN
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,NaN,0.4466,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1615814640000,1.3011,1.3037,1.2975,1.2979,74831.67,0.001774,1.2979,0,0.0,1.216610,3.294016
99996,1615814700000,1.2974,1.2977,1.2878,1.2936,126564.87,0.001323,1.2936,0,0.0,1.216667,3.294016
99997,1615814760000,1.2931,1.3007,1.2918,1.2951,97583.17,0.001077,1.2931,0,0.0,1.219505,3.294016
99998,1615814820000,1.2961,1.2961,1.2900,1.2917,126217.50,0.000658,1.2917,0,0.0,1.231599,3.294016


In [187]:
# df_json_small['test'] = df_json_small[['date', 'high']].rolling(window=2, method='table').apply(test, raw=True, engine='numba')
# df_test = df_json_small[['date', 'high']].rolling(window=2, method='table').apply(test, raw=True, engine='numba').copy()
df_test = df_json_small[['date', 'high']].rolling(window=3, method='table').apply(test, raw=True, engine='numba')['date'].copy()
# df_json_small
# df_json_small['test'] = df_test
df_test

[[1.6098048e+12 4.5070000e-01]] 1609804800000.0 0.45070000000000005 [1.6098048e+12]
[[1.60980480e+12 4.50700000e-01]
 [1.60980486e+12 4.53000000e-01]] 1609804860000.0 0.453 [1.60980480e+12 1.60980486e+12]
[[1.60980480e+12 4.50700000e-01]
 [1.60980486e+12 4.53000000e-01]
 [1.60980492e+12 4.49200000e-01]] 1609804920000.0 0.44920000000000004 [1.60980480e+12 1.60980486e+12 1.60980492e+12]
[[1.60980486e+12 4.53000000e-01]
 [1.60980492e+12 4.49200000e-01]
 [1.60980498e+12 4.47500000e-01]] 1609804980000.0 0.4475 [1.60980486e+12 1.60980492e+12 1.60980498e+12]
[[1.60980492e+12 4.49200000e-01]
 [1.60980498e+12 4.47500000e-01]
 [1.60980504e+12 4.55000000e-01]] 1609805040000.0 0.455 [1.60980492e+12 1.60980498e+12 1.60980504e+12]
[[1.60980498e+12 4.47500000e-01]
 [1.60980504e+12 4.55000000e-01]
 [1.60980510e+12 4.54500000e-01]] 1609805100000.0 0.4545 [1.60980498e+12 1.60980504e+12 1.60980510e+12]
[[1.60980504e+12 4.55000000e-01]
 [1.60980510e+12 4.54500000e-01]
 [1.60980516e+12 4.59200000e-01]] 160

0             NaN
1             NaN
2    7.231244e+11
3    7.203877e+11
4    7.324613e+11
5    7.316564e+11
6    7.392225e+11
7    7.405104e+11
8    7.421202e+11
9    7.419593e+11
Name: date, dtype: float64

In [100]:
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'B': [10, 20, 30, 40, 50, 60, 70, 80, 90]})

def my_custom_function(x):
    return x['A'] + x['B']

result = df.rolling(window=3).apply(my_custom_function, raw=True)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [54]:
start_time = datetime.now()
df_json_small['predict_price_boundary1'] = [predict_boundary_old(df_sub['average'].to_numpy(), df_sub['close'].to_numpy())
                                            for df_sub in df_json_small[['average', 'close']].rolling(timewindow_for_predict)]
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 12043.616 ms


/tmp/ipykernel_118777/1073477144.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_json_small['predict_price_boundary1'] = [predict_boundary_old(df_sub['average'].to_numpy(), df_sub['close'].to_numpy())


In [30]:
start_time = datetime.now()
# df_json_small['predict'] = df_json_small['average'].rolling(window=timewindow_for_predict).apply(predict_price_boundary, args=(df_json_small['close'], 1, 11), raw=True)
# df_json['predict'] = (
#     df_json[['average', 'close']]
#     .rolling(window=timewindow_for_predict, method='table')
#     .apply(predict_boundary, raw=True, engine='numba')['average']
#     .copy()
# )

df_json_small['predict'] = (
    df_json_small[['average', 'close']]
    .rolling(window=timewindow_for_predict, method='table')
    .apply(predict_boundary, raw=True, engine='numba')['average']
    .copy()
)
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 402.61400000000003 ms


In [82]:
df_json_small

,date,open,high,low,close,volume,macdhist,low_value,test_dw_1,test_dw_2,average,predict
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,NaN,0.4465,0,NaN,NaN,NaN
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,NaN,0.4467,0,NaN,NaN,NaN
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,NaN,0.4464,0,NaN,NaN,NaN
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,NaN,0.4460,0,NaN,NaN,NaN
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,NaN,0.4466,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1615814640000,1.3011,1.3037,1.2975,1.2979,74831.67,0.001774,1.2979,0,0.0,1.216610,3.309037
99996,1615814700000,1.2974,1.2977,1.2878,1.2936,126564.87,0.001323,1.2936,0,0.0,1.216667,3.309037
99997,1615814760000,1.2931,1.3007,1.2918,1.2951,97583.17,0.001077,1.2931,0,0.0,1.219505,3.309037
99998,1615814820000,1.2961,1.2961,1.2900,1.2917,126217.50,0.000658,1.2917,0,0.0,1.231599,3.309037


In [31]:
df_json

,date,open,high,low,close,volume,macdhist,low_value,average,predict
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,NaN,0.4465,NaN,NaN
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,NaN,0.4467,NaN,NaN
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,NaN,0.4464,NaN,NaN
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,NaN,0.4460,NaN,NaN
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,NaN,0.4466,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
918001,1664944440000,0.3525,0.3525,0.3524,0.3525,5655.00,-0.000060,0.3525,0.351049,3.0
918002,1664944500000,0.3524,0.3525,0.3524,0.3525,3878.00,-0.000047,0.3524,0.351131,3.0
918003,1664944560000,0.3525,0.3526,0.3525,0.3526,18512.00,-0.000027,0.3525,0.351155,3.0
918004,1664944620000,0.3527,0.3528,0.3527,0.3527,3693.00,-0.000005,0.3527,0.351189,3.0


## Rolling and detectwave

In [11]:
df_json[:-10]['high']

0         0.4507
1         0.4530
2         0.4492
3         0.4475
4         0.4550
           ...  
917991    0.3531
917992    0.3531
917993    0.3528
917994    0.3531
917995    0.3531
Name: high, Length: 917996, dtype: float64

In [23]:
df_json['mean'] = 0
df_json.loc[(df_json['high'] > 0), 'mean'] = 1

In [19]:
df_json

,date,open,high,low,close,volume,mean
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,1
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,1
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,1
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,1
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,1
...,...,...,...,...,...,...,...
918001,1664944440000,0.3525,0.3525,0.3524,0.3525,5655.00,1
918002,1664944500000,0.3524,0.3525,0.3524,0.3525,3878.00,1
918003,1664944560000,0.3525,0.3526,0.3525,0.3526,18512.00,1
918004,1664944620000,0.3527,0.3528,0.3527,0.3527,3693.00,1


In [4]:
values = df_json.values

In [5]:
print(values)

[[1.60980480e+12 4.46500000e-01 4.50700000e-01 4.46300000e-01
  4.50000000e-01 6.67946650e+05]
 [1.60980486e+12 4.49600000e-01 4.53000000e-01 4.43500000e-01
  4.46700000e-01 4.99775210e+05]
 [1.60980492e+12 4.47100000e-01 4.49200000e-01 4.45500000e-01
  4.46400000e-01 2.90446520e+05]
 ...
 [1.66494456e+12 3.52500000e-01 3.52600000e-01 3.52500000e-01
  3.52600000e-01 1.85120000e+04]
 [1.66494462e+12 3.52700000e-01 3.52800000e-01 3.52700000e-01
  3.52700000e-01 3.69300000e+03]
 [1.66494468e+12 3.52700000e-01 3.52700000e-01 3.52600000e-01
  3.52600000e-01 1.11000000e+02]]


In [79]:
def roll(df, w, **kwargs):
    v = df.values
    d0, d1 = v.shape
    s0, s1 = v.strides

    a = np.lib.stride_tricks.as_strided(v, (d0 - (w - 1), w, d1), (s0, s0, s1))

    rolled_df = pd.concat({
        row: pd.DataFrame(values, columns=df.columns)
        for row, values in zip(range(w-1, d0), a)
    })

    return rolled_df.groupby(level=0, **kwargs)

In [12]:
time_window = 10

start_time = datetime.now()
result = roll(df_json, time_window).mean()
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")
print(result)

Matched filter took 33366.399000000005 ms
                   0        1        2        3        4           5
9       1.609805e+12  0.45185  0.45510  0.44915  0.45323  411080.754
10      1.609805e+12  0.45321  0.45673  0.45051  0.45480  369864.301
11      1.609805e+12  0.45492  0.45810  0.45182  0.45579  353410.383
12      1.609805e+12  0.45593  0.45984  0.45270  0.45758  346658.463
13      1.609805e+12  0.45778  0.46164  0.45448  0.45902  331045.395
...              ...      ...      ...      ...      ...         ...
918001  1.664944e+12  0.35270  0.35280  0.35251  0.35266   26278.300
918002  1.664944e+12  0.35263  0.35274  0.35248  0.35263   15693.900
918003  1.664944e+12  0.35260  0.35272  0.35245  0.35261   17173.000
918004  1.664944e+12  0.35259  0.35269  0.35246  0.35257   15646.500
918005  1.664944e+12  0.35255  0.35265  0.35245  0.35255   13351.600

[917997 rows x 6 columns]


In [24]:
# def roll_gpt(df, w, **kwargs):
#     rolled_df = pd.concat([df.shift(-i) for i in range(w)], axis=1)
#     rolled_df.columns = [f'window_{i}' for i in range(w)]
#     return rolled_df.dropna().groupby(**kwargs)

# def roll_gpt(df, w, **kwargs):
#     n_cols = df.shape[1]
#     rolled_df = pd.concat([df.shift(-i) for i in range(w)], axis=1)
#     rolled_df.columns = [f'window_{i}_{col}' for i in range(w) for col in df.columns]
#     return rolled_df.dropna().groupby(**kwargs)

def roll_gpt(df, w, **kwargs):
    n_cols = df.shape[1]
    rolled_df = pd.concat([df.shift(-i) for i in range(w)], axis=1)
    rolled_df["rolling_window"] = [i for i in range(w)]
    return rolled_df.dropna().groupby(**kwargs)

def roll_v2(df, w, **kwargs):
    rolled_df = df.rolling(w).apply(lambda x: x)
    return rolled_df.dropna().groupby(**kwargs)

In [26]:
time_window = 10

start_time = datetime.now()
# result = roll_v2(df_json, time_window).mean()
result = df_json.rolling(time_window).mean()

print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")
print(result)

Matched filter took 61.843 ms
                   0        1        2        3        4        5
0                NaN      NaN      NaN      NaN      NaN      NaN
1                NaN      NaN      NaN      NaN      NaN      NaN
2                NaN      NaN      NaN      NaN      NaN      NaN
3                NaN      NaN      NaN      NaN      NaN      NaN
4                NaN      NaN      NaN      NaN      NaN      NaN
...              ...      ...      ...      ...      ...      ...
918001  1.664944e+12  0.35270  0.35280  0.35251  0.35266  26278.3
918002  1.664944e+12  0.35263  0.35274  0.35248  0.35263  15693.9
918003  1.664944e+12  0.35260  0.35272  0.35245  0.35261  17173.0
918004  1.664944e+12  0.35259  0.35269  0.35246  0.35257  15646.5
918005  1.664944e+12  0.35255  0.35265  0.35245  0.35255  13351.6

[918006 rows x 6 columns]


In [194]:
df_json = df_json.fillna(0)
df_json

,date,open,high,low,close,volume,macdhist,low_value
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,0.000000,0.4465
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,0.000000,0.4467
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,0.000000,0.4464
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,0.000000,0.4460
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,0.000000,0.4466
...,...,...,...,...,...,...,...,...
918001,1664944440000,0.3525,0.3525,0.3524,0.3525,5655.00,-0.000060,0.3525
918002,1664944500000,0.3524,0.3525,0.3524,0.3525,3878.00,-0.000047,0.3524
918003,1664944560000,0.3525,0.3526,0.3525,0.3526,18512.00,-0.000027,0.3525
918004,1664944620000,0.3527,0.3528,0.3527,0.3527,3693.00,-0.000005,0.3527


In [20]:
hist = df_json['macdhist'].to_numpy()

In [21]:
print(hist)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.53624207e-05
 5.48898722e-05 5.38129077e-05]


In [30]:
df_json

,date,open,high,low,close,volume,macdhist,low_value
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,0.000000,0.4465
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,0.000000,0.4467
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,0.000000,0.4464
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,0.000000,0.4460
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,0.000000,0.4466
...,...,...,...,...,...,...,...,...
918001,1664944440000,0.3525,0.3525,0.3524,0.3525,5655.00,0.000007,0.3525
918002,1664944500000,0.3524,0.3525,0.3524,0.3525,3878.00,0.000017,0.3524
918003,1664944560000,0.3525,0.3526,0.3525,0.3526,18512.00,0.000035,0.3525
918004,1664944620000,0.3527,0.3528,0.3527,0.3527,3693.00,0.000055,0.3527


In [83]:
def custom_function(x):
    # print(x)
    # return x['low'] * x['open'] + x['high']
    return 1

# window_size = 3


In [76]:
print(len(df_json))
result = np.ndarray((len(df_json)))
print(np.shape(result))

918006
(918006,)


In [85]:
start_time = datetime.now()
# test_result = roll(df_json, window_size).apply(detect_wave).fillna(0)
test_result = roll(df_json, window_size).apply(custom_function).fillna(0)
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 84126.154 ms


In [29]:
window_size = 200
toler_hist = 1.0
toler_low = 0.99
# result = df_json.rolling(window_size).apply(lambda x: custom_function(x))
# result = df_json.rolling(window_size).apply(lambda x: custom_function(x))
# result = df_json.shift(-window_size+1).rolling(window_size).apply(custom_function)

In [77]:

start_time = datetime.now()

k = 0
result = np.ndarray(len(df_json))
for df_sub in df_json.rolling(window_size):
    # print(custom_function(df_sub))
    # print(detect_wave(df_sub))
    result[k] = detect_wave(df_sub)
    k += 1
    # if k > 100:
    #     break

print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")


Function took 57380.045000000006 ms


In [220]:
start_time = datetime.now()
res_test2 = [detect_wave(df_sub) for df_sub in df_json.rolling(window_size)]
# res_test2 = [custom_function(df_sub) for df_sub in df_json.rolling(window_size)]
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")


Function took 101068.601 ms


In [19]:
start_time = datetime.now()
# res_test2 = [detect_wave_new(df_sub, window_size) for df_sub in df_json.rolling(window_size)]
# res_test2 = [detect_wave_numpy(df_sub['macdhist'].to_numpy(), df_sub['low_value'].to_numpy(), window_size) for df_sub in df_json.rolling(window_size)]
res_test2 = [detect_wave(df_sub['macdhist'].to_numpy(), df_sub['low_value'].to_numpy()) for df_sub in df_json[['macdhist', 'low_value']].rolling(window_size)]
# res_test2 = [custom_function(df_sub) for df_sub in df_json.rolling(window_size)]
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 111693.87299999999 ms


In [34]:
@numba.jit(nopython=True)
def detect_wave(hist, low, toler_hist=1.00, toler_low=0.99, n_wave=3):
    # hist = df['macdhist'].to_numpy()
    # low = df["low_value"].to_numpy()
    n = hist.shape[0]

    if hist[-1] > 0:
        return 0

    wave_list = []
    hist_min = hist[-1]
    low_min = low[-1]

    # start_time = datetime.now()
    for i in range(2, n):
        hist_min = min(hist_min, hist[-i])
        low_min = min(low_min, low[-i])

        #Конец волны
        if hist[-i] > 0 and hist[-i+1] < 0:
            wave_list.append([hist_min, low_min])

            if len(wave_list) == n_wave:
                break

        #Начало волны
        if hist[-i] < 0 and hist[-i+1] > 0:
            hist_min = hist[-i]
            low_min = low[-i]

    # print("Waves search took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

    if len(wave_list) > 1:

        if ( (wave_list[0][0] > (wave_list[1][0] * toler_hist)) & (wave_list[0][1] < (wave_list[1][1]) * toler_low) ):
            return 1

        # if ( (wave_list[0][0] > (wave_list[2][0] * toler_hist)) & (wave_list[0][1] < (wave_list[2][1]) * toler_low) ):
        #     return 2

    else:
        return 0

    return 0


@numba.jit(nopython=True)
def detect_wave_optimal(data, toler_hist=1.00, toler_low=0.99, n_wave=3):
    hist = data[:, 0]
    low = data[:, 1]
    # hist = df['macdhist'].to_numpy()
    # low = df["low_value"].to_numpy()
    n = hist.shape[0]

    if hist[-1] > 0:
        return [0, low[-1]]

    wave_list = []
    hist_min = hist[-1]
    low_min = low[-1]

    # start_time = datetime.now()
    for i in range(2, n):
        hist_min = min(hist_min, hist[-i])
        low_min = min(low_min, low[-i])

        #Конец волны
        if hist[-i] > 0 and hist[-i+1] < 0:
            wave_list.append([hist_min, low_min])

            if len(wave_list) == n_wave:
                break

        #Начало волны
        if hist[-i] < 0 and hist[-i+1] > 0:
            hist_min = hist[-i]
            low_min = low[-i]

    # print("Waves search took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

    if len(wave_list) > 1:

        if ( (wave_list[0][0] > (wave_list[1][0] * toler_hist)) & (wave_list[0][1] < (wave_list[1][1]) * toler_low) ):
            return [1, low[-1]]

        # if ( (wave_list[0][0] > (wave_list[2][0] * toler_hist)) & (wave_list[0][1] < (wave_list[2][1]) * toler_low) ):
        #     return 2

    else:
        return [0, low[-1]]

    return [0, low[-1]]

In [40]:
window_size = 200
start_time = datetime.now()
df_test = df_json[['macdhist', 'low_value']].rolling(window=window_size, method='table').apply(detect_wave_optimal, raw=True, engine='numba')['macdhist'].copy()
print("Function took", (datetime.now() - start_time).total_seconds() * 1000, "ms")

Function took 1114.246 ms


In [33]:
df_json['test_dw_1'] = res_test2
df_json['test_dw_2'] = df_test
df_json

,date,open,high,low,close,volume,macdhist,low_value,test_dw_1,test_dw_2
0,1609804800000,0.4465,0.4507,0.4463,0.4500,667946.65,NaN,0.4465,0,NaN
1,1609804860000,0.4496,0.4530,0.4435,0.4467,499775.21,NaN,0.4467,0,NaN
2,1609804920000,0.4471,0.4492,0.4455,0.4464,290446.52,NaN,0.4464,0,NaN
3,1609804980000,0.4460,0.4475,0.4424,0.4464,324858.12,NaN,0.4460,0,NaN
4,1609805040000,0.4466,0.4550,0.4445,0.4537,421508.95,NaN,0.4466,0,NaN
...,...,...,...,...,...,...,...,...,...,...
918001,1664944440000,0.3525,0.3525,0.3524,0.3525,5655.00,-0.000060,0.3525,0,0.0
918002,1664944500000,0.3524,0.3525,0.3524,0.3525,3878.00,-0.000047,0.3524,0,0.0
918003,1664944560000,0.3525,0.3526,0.3525,0.3526,18512.00,-0.000027,0.3525,0,0.0
918004,1664944620000,0.3527,0.3528,0.3527,0.3527,3693.00,-0.000005,0.3527,0,0.0


In [41]:
# x = PrettyTable()
a = np.random.randint(-10, 10, 20)
# x.add_row([i for i in range(20)])
# x.add_row(a)
# print(x)
# print([i for i in range(20)])
print(a)
# print(a[:-1])
ind_neg = np.where(a < 0)[0]
# print(b, a[b])
# ind_neg = b[0]
print(ind_neg)

print()

[  6   5  -3   0  -9  -5  -3  -7 -10 -10  -9  -5   5  -4  -6   5  -3   3
   5   3]
[ 2  4  5  6  7  8  9 10 11 13 14 16]



In [43]:
for k in range(-1, -len(ind_neg[ind_left])-1, -1):
    print(a[ind_neg[ind_left]][k], a[ind_neg[ind_right]][k])
    print(a[ind_neg[ind_left][k]:ind_neg[ind_right][k]+1], np.min(a[ind_neg[ind_left][k]:ind_neg[ind_right][k]+1]))

print(len(ind_neg[ind_left]), len(ind_neg[ind_right]))

-3 -3
[-3] -3
-4 -6
[-4 -6] -6
-9 -5
[ -9  -5  -3  -7 -10 -10  -9  -5] -10
-3 -3
[-3] -3
4 4


In [42]:

ind_right = np.where((ind_neg[1:] - ind_neg[:-1]) > 1)
ind_right = np.concatenate([ind_right[0], [len(ind_neg)-1]])
print(ind_right)
print(ind_neg[ind_right])
print(a[ind_neg[ind_right]])

ind_left = ind_right[:-1] + 1
ind_left = np.concatenate([[0], ind_left])
print(ind_left)
print(ind_neg[ind_left])
print(a[ind_neg[ind_left]])

print()


# print()
# ind_left = ind_right[0] - 1
# if ind_left[0] < 0:
#     ind_left[0] = 0
# print(ind_left)
# print(c[ind_left])
# print(a[c[ind_left]])


[ 0  8 10 11]
[ 2 11 14 16]
[-3 -5 -6 -3]
[ 0  1  9 11]
[ 2  4 13 16]
[-3 -9 -4 -3]

